In [5]:
# organize imports
import cv2
import imutils
import numpy as np

# global variables
bg = None

In [6]:
#--------------------------------------------------
# To find the running average over the background
#--------------------------------------------------
def run_avg(image, aWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

In [19]:
#---------------------------------------------
# To segment the region of hand in the image
#---------------------------------------------
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts,_) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

In [32]:
#-----------------
# MAIN FUNCTION
#-----------------
if __name__ == "__main__":
    # initialize weight for running average
    aWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(1)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0

    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width=700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                cv2.imshow("Thesholded", thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break

# free up memory
camera.release()
cv2.destroyAllWindows()

In [1]:
#------------------------------------------------------------
# SEGMENT, RECOGNIZE and COUNT fingers from a video sequence
#------------------------------------------------------------

# organize imports
import cv2
import imutils
import numpy as np
from sklearn.metrics import pairwise

# global variables
bg = None

#--------------------------------------------------
# To find the running average over the background
#--------------------------------------------------
def run_avg(image, accumWeight):
    global bg
    # initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # compute weighted average, accumulate it and update the background
    cv2.accumulateWeighted(image, bg, accumWeight)

#---------------------------------------------
# To segment the region of hand in the image
#---------------------------------------------
def segment(image, threshold=25):
    global bg
    # find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

#--------------------------------------------------------------
# To count the number of fingers in the segmented hand region
#--------------------------------------------------------------
def count(thresholded, segmented):
    # find the convex hull of the segmented hand region
    chull = cv2.convexHull(segmented)

    # find the most extreme points in the convex hull
    extreme_top    = tuple(chull[chull[:, :, 1].argmin()][0])
    extreme_bottom = tuple(chull[chull[:, :, 1].argmax()][0])
    extreme_left   = tuple(chull[chull[:, :, 0].argmin()][0])
    extreme_right  = tuple(chull[chull[:, :, 0].argmax()][0])

    # find the center of the palm
    cX = int((extreme_left[0] + extreme_right[0]) / 2)
    cY = int((extreme_top[1] + extreme_bottom[1]) / 2)

    # find the maximum euclidean distance between the center of the palm
    # and the most extreme points of the convex hull
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[extreme_left, extreme_right, extreme_top, extreme_bottom])[0]
    maximum_distance = distance[distance.argmax()]

    # calculate the radius of the circle with 80% of the max euclidean distance obtained
    radius = int(0.8 * maximum_distance)

    # find the circumference of the circle
    circumference = (2 * np.pi * radius)

    # take out the circular region of interest which has 
    # the palm and the fingers
    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
	
    # draw the circular ROI
    cv2.circle(circular_roi, (cX, cY), radius, 255, 1)

    # take bit-wise AND between thresholded hand using the circular ROI as the mask
    # which gives the cuts obtained using mask on the thresholded hand image
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

    # compute the contours in the circular ROI
    (cnts, _) = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # initalize the finger count
    count = 0

    # loop through the contours found
    for c in cnts:
        # compute the bounding box of the contour
        (x, y, w, h) = cv2.boundingRect(c)

        # increment the count of fingers only if -
        # 1. The contour region is not the wrist (bottom area)
        # 2. The number of points along the contour does not exceed
        #     25% of the circumference of the circular ROI
        if ((cY + (cY * 0.25)) > (y + h)) and ((circumference * 0.25) > c.shape[0]):
            count += 1

    return count

#-----------------
# MAIN FUNCTION
#-----------------
if __name__ == "__main__":
    # initialize accumulated weight
    accumWeight = 0.5

    # get the reference to the webcam
    camera = cv2.VideoCapture(1)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # initialize num of frames
    num_frames = 0

    # calibration indicator
    calibrated = False

    # keep looping, until interrupted
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width=700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our weighted average model gets calibrated
        if num_frames < 30:
            run_avg(gray, accumWeight)
            if num_frames == 1:
                print("[STATUS] please wait! calibrating...")
            elif num_frames == 29:
                print("[STATUS] calibration successfull...")
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))

                # count the number of fingers
                fingers = count(thresholded, segmented)

                cv2.putText(clone, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                
                # show the thresholded image
                cv2.imshow("Thesholded", thresholded)

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break

# free up memory
camera.release()
cv2.destroyAllWindows()

[STATUS] please wait! calibrating...
[STATUS] calibration successfull...


In [2]:
"""
In this section, I will experiment with finger tracking instead
"""
import cv2
import numpy as np

traverse_point = []
total_rectangle = 9
hand_rect_one_x = None
hand_rect_one_y = None

hand_rect_two_x = None
hand_rect_two_y = None


def rescale_frame(frame, wpercent=130, hpercent=130):
    width = int(frame.shape[1] * wpercent / 100)
    height = int(frame.shape[0] * hpercent / 100)
    return cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)


def contours(hist_mask_image):
    gray_hist_mask_image = cv2.cvtColor(hist_mask_image, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray_hist_mask_image, 0, 255, 0)
    cont, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return cont


def max_contour(contour_list):
    max_i = 0
    max_area = 0

    for i in range(len(contour_list)):
        cnt = contour_list[i]

        area_cnt = cv2.contourArea(cnt)

        if area_cnt > max_area:
            max_area = area_cnt
            max_i = i

        return contour_list[max_i]


def draw_rect(frame):
    rows, cols, _ = frame.shape
    global total_rectangle, hand_rect_one_x, hand_rect_one_y, hand_rect_two_x, hand_rect_two_y

    hand_rect_one_x = np.array(
        [6 * rows / 20, 6 * rows / 20, 6 * rows / 20, 9 * rows / 20, 9 * rows / 20, 9 * rows / 20, 12 * rows / 20,
         12 * rows / 20, 12 * rows / 20], dtype=np.uint32)

    hand_rect_one_y = np.array(
        [9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20,
         10 * cols / 20, 11 * cols / 20], dtype=np.uint32)

    hand_rect_two_x = hand_rect_one_x + 10
    hand_rect_two_y = hand_rect_one_y + 10

    for i in range(total_rectangle):
        cv2.rectangle(frame, (hand_rect_one_y[i], hand_rect_one_x[i]),
                      (hand_rect_two_y[i], hand_rect_two_x[i]),
                      (0, 255, 0), 1)

    return frame


def hand_histogram(frame):
    global hand_rect_one_x, hand_rect_one_y
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    roi = np.zeros([90, 10, 3], dtype=hsv_frame.dtype)    
    
    for i in range(total_rectangle):
        roi[i * 10: i * 10 + 10, 0: 10] = hsv_frame[hand_rect_one_x[i]:hand_rect_one_x[i] + 10,
                                          hand_rect_one_y[i]:hand_rect_one_y[i] + 10]

    hand_hist = cv2.calcHist([roi], [0, 1], None, [180, 256], [0, 180, 0, 256])
    return cv2.normalize(hand_hist, hand_hist, 0, 255, cv2.NORM_MINMAX)


def hist_masking(frame, hist):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    dst = cv2.calcBackProject([hsv], [0, 1], hist, [0, 180, 0, 256], 1)

    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (31, 31))
    cv2.filter2D(dst, -1, disc, dst)

    ret, thresh = cv2.threshold(dst, 150, 255, cv2.THRESH_BINARY)

    # thresh = cv2.dilate(thresh, None, iterations=5)

    thresh = cv2.merge((thresh, thresh, thresh))

    return cv2.bitwise_and(frame, thresh)


def centroid(max_contour):
    moment = cv2.moments(max_contour)
    if moment['m00'] != 0:
        cx = int(moment['m10'] / moment['m00'])
        cy = int(moment['m01'] / moment['m00'])
        return cx, cy
    else:
        return None


def farthest_point(defects, contour, centroid):
    if defects is not None and centroid is not None:
        s = defects[:, 0][:, 0]
        cx, cy = centroid

        x = np.array(contour[s][:, 0][:, 0], dtype=np.float)
        y = np.array(contour[s][:, 0][:, 1], dtype=np.float)

        xp = cv2.pow(cv2.subtract(x, cx), 2)
        yp = cv2.pow(cv2.subtract(y, cy), 2)
        dist = cv2.sqrt(cv2.add(xp, yp))

        dist_max_i = np.argmax(dist)

        if dist_max_i < len(s):
            farthest_defect = s[dist_max_i]
            farthest_point = tuple(contour[farthest_defect][0])
            return farthest_point
        else:
            return None


def draw_circles(frame, traverse_point):
    if traverse_point is not None:
        for i in range(len(traverse_point)):
            cv2.circle(frame, traverse_point[i], int(5 - (5 * i * 3) / 100), [0, 255, 255], -1)


def manage_image_opr(frame, hand_hist):
    hist_mask_image = hist_masking(frame, hand_hist)
    contour_list = contours(hist_mask_image)
    max_cont = max_contour(contour_list)

    cnt_centroid = centroid(max_cont)
    cv2.circle(frame, cnt_centroid, 5, [255, 0, 255], -1)

    if max_cont is not None:
        hull = cv2.convexHull(max_cont, returnPoints=False)
        defects = cv2.convexityDefects(max_cont, hull)
        far_point = farthest_point(defects, max_cont, cnt_centroid)
        print("Centroid : " + str(cnt_centroid) + ", farthest Point : " + str(far_point))
        cv2.circle(frame, far_point, 5, [0, 0, 255], -1)
        if len(traverse_point) < 20:
            traverse_point.append(far_point)
        else:
            traverse_point.pop(0)
            traverse_point.append(far_point)

        draw_circles(frame, traverse_point)
#-----------------
# MAIN FUNCTION
#-----------------
if __name__ == "__main__":
    global hand_hist
    is_hand_hist_created = False
    # get the reference to the webcam
    camera = cv2.VideoCapture(0)

    preprocessed = None
    num_frames = 0

    # keep looping, until interrupted
    while(camera.isOpened()):
        pressed_key = cv2.waitKey(1)
        # get the current frame
        (grabbed, preprocessed) = camera.read()

        # flip the frame so that it is not the mirror view
        preprocessed = cv2.flip(preprocessed, 1)
        
        #Copy the frame to display preprocessed and post-processed image
        processed = preprocessed.copy()
        if pressed_key & 0xFF == ord('z'):
            is_hand_hist_created = True
            hand_hist = hand_histogram(processed)

        if is_hand_hist_created:
            manage_image_opr(processed, hand_hist)
        else:
            processed = draw_rect(processed) 




        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", preprocessed)
        cv2.imshow("Hand Feed", processed)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break

    # free up memory
    camera.release()
    cv2.destroyAllWindows()


Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (368, 478), farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (567, 478), farthest Point : None
Centroid : None, farthest Point : None
Centroid : (251, 476), farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (430, 478), farthest Point : None
Centroid : (503, 440), farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (238, 478), farthest Point : None
Centroid : (238, 478), farth

Centroid : None, farthest Point : None
Centroid : (188, 469), farthest Point : (191, 468)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (428, 477), farthest Point : (430, 475)
Centroid : (206, 469), farthest Point : None
Centroid : None, farthest Point : None
Centroid : (217, 478), farthest Point : (221, 479)
Centroid : None, farthest Point : None
Centroid : (444, 406), farthest Point : None
Centroid : (444, 406), farthest Point : None
Centroid : None, farthest Point : None
Centroid : (319, 236), farthest Point : (0, 479)
Centroid : (549, 472), farthest Point : None
Centroid : (232, 431), farthest Point : (235, 430)
Centroid : None, farthest Point : None
Centroid : (467, 478), farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None

Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (129, 478), farthest Point : None
Centroid : None, farthest Point : None
Centroid : (384, 477), farthest Point : None
Centroid : (387, 477), farthest Point : (389, 477)
Centroid : None, farthest Point : None
Centroid : (166, 478), farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (420, 416), farthest Point : None
Centroid : (567, 460), farthest Point : (588, 479)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (0, 464), farthest Point : (0, 469)
Centroid : None, farthest Point : None
Centroid : (551, 461), farthest Point : (566, 470)
Centroid : (479, 456), farthest Point : (480, 464)
Centroid : (479, 456), 

Centroid : (266, 476), farthest Point : (275, 479)
Centroid : None, farthest Point : None
Centroid : (258, 477), farthest Point : (263, 479)
Centroid : (258, 477), farthest Point : (263, 479)
Centroid : (256, 476), farthest Point : (263, 479)
Centroid : (257, 477), farthest Point : (263, 479)
Centroid : None, farthest Point : None
Centroid : (258, 476), farthest Point : (261, 473)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (421, 478), farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (465, 455), farthest Point : (443, 476)
Centroid : None, farthest Point : None
Centroid : (453, 403), farthest Point : None
Centroid : (537, 472), farthest Point : (555, 476)
Centroid : (580, 467), farthest Point : (593, 477)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (546, 472), farthest Point : (529, 479)
Centroid :

Centroid : (638, 358), farthest Point : (639, 356)
Centroid : None, farthest Point : None
Centroid : (552, 442), farthest Point : (550, 444)
Centroid : None, farthest Point : None
Centroid : (533, 441), farthest Point : None
Centroid : (591, 71), farthest Point : (590, 70)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (598, 472), farthest Point : (614, 476)
Centroid : (579, 11), farthest Point : None
Centroid : None, farthest Point : None
Centroid : (558, 443), farthest Point : (557, 448)
Centroid : (616, 451), farthest Point : (631, 448)
Centroid : None, farthest Point : None
Centroid : (548, 443), farthest Point : None
Centroid : (638, 30), farthest Point : None
Centroid : None, farthest Point : None
Centroid : (631, 18), farthest Point : (634, 43)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
C

Centroid : (316, 241), farthest Point : (639, 479)
Centroid : (316, 241), farthest Point : (639, 2)
Centroid : (316, 241), farthest Point : (639, 2)
Centroid : (316, 241), farthest Point : (639, 2)
Centroid : (316, 241), farthest Point : (639, 0)
Centroid : (316, 241), farthest Point : (639, 0)
Centroid : (317, 241), farthest Point : (639, 0)
Centroid : (317, 241), farthest Point : (639, 0)
Centroid : None, farthest Point : None
Centroid : (317, 240), farthest Point : (639, 0)
Centroid : None, farthest Point : None
Centroid : (317, 240), farthest Point : (639, 0)
Centroid : (317, 240), farthest Point : (639, 479)
Centroid : (317, 240), farthest Point : (639, 479)
Centroid : (627, 11), farthest Point : (627, 25)
Centroid : None, farthest Point : None
Centroid : (632, 19), farthest Point : (627, 29)
Centroid : None, farthest Point : None
Centroid : (318, 240), farthest Point : (639, 0)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest

Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (598, 477), farthest Point : (594, 479)
Centroid : (592, 478), farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (592, 478), farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (601, 476), farthest Point : (600, 478)
Centroid : (598, 477), farthest Point : (605, 476)
Centroid : None, farthest Point : None
Centroid : (637, 477), farthest Point : None
Centroid : (398, 450), farthest Point : None
Centroid : (512, 378), farthest Point : (515, 378)
Centroid : (526, 416), farthest Point : (520, 422)
Centroid : (2, 478), farthest Point : (6, 478)
Centroid : 

Centroid : (231, 476), farthest Point : None
Centroid : (231, 476), farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (337, 400), farthest Point : (340, 409)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (152, 305), farthest Point : (145, 313)
Centroid : (473, 222), farthest Point : (475, 224)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (104, 478), farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (49, 223), farthest Point : (43, 228)
Centroid : (637, 478), farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (113, 478), farthest Point : None
Centroid : (0, 192), farthest Point : None
Centroid : None, farthe

Centroid : None, farthest Point : None
Centroid : (316, 241), farthest Point : (639, 0)
Centroid : (316, 241), farthest Point : (639, 0)
Centroid : (316, 241), farthest Point : (639, 0)
Centroid : (316, 241), farthest Point : (639, 0)
Centroid : None, farthest Point : None
Centroid : (317, 241), farthest Point : (639, 0)
Centroid : (581, 7), farthest Point : (591, 4)
Centroid : (577, 21), farthest Point : None
Centroid : (578, 6), farthest Point : (575, 19)
Centroid : (586, 24), farthest Point : None
Centroid : (317, 240), farthest Point : (639, 0)
Centroid : (318, 240), farthest Point : (639, 0)
Centroid : (587, 1), farthest Point : (586, 4)
Centroid : None, farthest Point : None
Centroid : (319, 239), farthest Point : (639, 0)
Centroid : (318, 240), farthest Point : (639, 0)
Centroid : (319, 239), farthest Point : (639, 479)
Centroid : (319, 239), farthest Point : (639, 0)
Centroid : (319, 239), farthest Point : (638, 0)
Centroid : None, farthest Point : None
Centroid : (319, 239), f

In [2]:
import cv2
import numpy as np

hand_hist = None
traverse_point = []
total_rectangle = 9
hand_rect_one_x = None
hand_rect_one_y = None

hand_rect_two_x = None
hand_rect_two_y = None


def rescale_frame(frame, wpercent=130, hpercent=130):
    width = int(frame.shape[1] * wpercent / 100)
    height = int(frame.shape[0] * hpercent / 100)
    return cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)


def contours(hist_mask_image):
    gray_hist_mask_image = cv2.cvtColor(hist_mask_image, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray_hist_mask_image, 0, 255, 0)
    cont, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return cont


def max_contour(contour_list):
    max_i = 0
    max_area = 0

    for i in range(len(contour_list)):
        cnt = contour_list[i]

        area_cnt = cv2.contourArea(cnt)

        if area_cnt > max_area:
            max_area = area_cnt
            max_i = i

        return contour_list[max_i]


def draw_rect(frame):
    rows, cols, _ = frame.shape
    global total_rectangle, hand_rect_one_x, hand_rect_one_y, hand_rect_two_x, hand_rect_two_y

    hand_rect_one_x = np.array(
        [6 * rows / 20, 6 * rows / 20, 6 * rows / 20, 9 * rows / 20, 9 * rows / 20, 9 * rows / 20, 12 * rows / 20,
         12 * rows / 20, 12 * rows / 20], dtype=np.uint32)

    hand_rect_one_y = np.array(
        [9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20, 10 * cols / 20, 11 * cols / 20, 9 * cols / 20,
         10 * cols / 20, 11 * cols / 20], dtype=np.uint32)

    hand_rect_two_x = hand_rect_one_x + 10
    hand_rect_two_y = hand_rect_one_y + 10

    for i in range(total_rectangle):
        cv2.rectangle(frame, (hand_rect_one_y[i], hand_rect_one_x[i]),
                      (hand_rect_two_y[i], hand_rect_two_x[i]),
                      (0, 255, 0), 1)

    return frame


def hand_histogram(frame):
    global hand_rect_one_x, hand_rect_one_y

    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    roi = np.zeros([90, 10, 3], dtype=hsv_frame.dtype)

    for i in range(total_rectangle):
        roi[i * 10: i * 10 + 10, 0: 10] = hsv_frame[hand_rect_one_x[i]:hand_rect_one_x[i] + 10,
                                          hand_rect_one_y[i]:hand_rect_one_y[i] + 10]

    hand_hist = cv2.calcHist([roi], [0, 1], None, [180, 256], [0, 180, 0, 256])
    return cv2.normalize(hand_hist, hand_hist, 0, 255, cv2.NORM_MINMAX)


def hist_masking(frame, hist):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    dst = cv2.calcBackProject([hsv], [0, 1], hist, [0, 180, 0, 256], 1)

    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (31, 31))
    cv2.filter2D(dst, -1, disc, dst)

    ret, thresh = cv2.threshold(dst, 150, 255, cv2.THRESH_BINARY)

    # thresh = cv2.dilate(thresh, None, iterations=5)

    thresh = cv2.merge((thresh, thresh, thresh))

    return cv2.bitwise_and(frame, thresh)


def centroid(max_contour):
    moment = cv2.moments(max_contour)
    if moment['m00'] != 0:
        cx = int(moment['m10'] / moment['m00'])
        cy = int(moment['m01'] / moment['m00'])
        return cx, cy
    else:
        return None


def farthest_point(defects, contour, centroid):
    if defects is not None and centroid is not None:
        s = defects[:, 0][:, 0]
        cx, cy = centroid

        x = np.array(contour[s][:, 0][:, 0], dtype=np.float)
        y = np.array(contour[s][:, 0][:, 1], dtype=np.float)

        xp = cv2.pow(cv2.subtract(x, cx), 2)
        yp = cv2.pow(cv2.subtract(y, cy), 2)
        dist = cv2.sqrt(cv2.add(xp, yp))

        dist_max_i = np.argmax(dist)

        if dist_max_i < len(s):
            farthest_defect = s[dist_max_i]
            farthest_point = tuple(contour[farthest_defect][0])
            return farthest_point
        else:
            return None


def draw_circles(frame, traverse_point):
    if traverse_point is not None:
        for i in range(len(traverse_point)):
            cv2.circle(frame, traverse_point[i], int(5 - (5 * i * 3) / 100), [0, 255, 255], -1)


def manage_image_opr(frame, hand_hist):
    hist_mask_image = hist_masking(frame, hand_hist)
    contour_list = contours(hist_mask_image)
    max_cont = max_contour(contour_list)

    cnt_centroid = centroid(max_cont)
    cv2.circle(frame, cnt_centroid, 5, [255, 0, 255], -1)

    if max_cont is not None:
        hull = cv2.convexHull(max_cont, returnPoints=False)
        defects = cv2.convexityDefects(max_cont, hull)
        far_point = farthest_point(defects, max_cont, cnt_centroid)
        print("Centroid : " + str(cnt_centroid) + ", farthest Point : " + str(far_point))
        cv2.circle(frame, far_point, 5, [0, 0, 255], -1)
        if len(traverse_point) < 20:
            traverse_point.append(far_point)
        else:
            traverse_point.pop(0)
            traverse_point.append(far_point)

        draw_circles(frame, traverse_point)


def main():
    global hand_hist
    is_hand_hist_created = False
    capture = cv2.VideoCapture(1)

    while capture.isOpened():
        pressed_key = cv2.waitKey(1)
        _, frame = capture.read()

        if pressed_key & 0xFF == ord('z'):
            is_hand_hist_created = True
            hand_hist = hand_histogram(frame)

        if is_hand_hist_created:
            manage_image_opr(frame, hand_hist)
            cv2.imshow("Contours", hist_masking(frame, hand_hist))


        else:
            frame = draw_rect(frame)

        cv2.imshow("Live Feed", rescale_frame(frame))

        if pressed_key == 27 or pressed_key == ord('q'):
            break

    cv2.destroyAllWindows()
    capture.release()


if __name__ == '__main__':
    main()


Centroid : (59, 398), farthest Point : (63, 390)
Centroid : (59, 398), farthest Point : (63, 390)
Centroid : (47, 433), farthest Point : (53, 430)
Centroid : (47, 433), farthest Point : (53, 430)
Centroid : (57, 437), farthest Point : (56, 438)
Centroid : (57, 437), farthest Point : (56, 438)
Centroid : (98, 428), farthest Point : (92, 441)
Centroid : (98, 428), farthest Point : (92, 441)
Centroid : (332, 241), farthest Point : (156, 0)
Centroid : (332, 241), farthest Point : (156, 0)
Centroid : (34, 478), farthest Point : (39, 477)
Centroid : (34, 478), farthest Point : (39, 477)
Centroid : (51, 387), farthest Point : (64, 383)
Centroid : (51, 387), farthest Point : (64, 383)
Centroid : (93, 347), farthest Point : (98, 340)
Centroid : (93, 347), farthest Point : (98, 340)
Centroid : (470, 444), farthest Point : (474, 453)
Centroid : (470, 444), farthest Point : (474, 453)
Centroid : (70, 427), farthest Point : (69, 430)
Centroid : (70, 427), farthest Point : (69, 430)
Centroid : None,

Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (572, 475), farthest Point : (555, 476)
Centroid : (572, 475), farthest Point : (555, 476)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (629, 472), farthest Point : (615, 479)
Centroid : (629, 472), farthest Point : (615, 479)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (597, 440), farthest Point : (599, 449)
Centroid : (597, 440), farthest Point : (599, 449)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (462, 473), farthest Point : (461, 475)
Centroid : (462, 473), farthest Point : (461, 475)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroi

Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (554, 447), farthest Point : None
Centroid : (554, 447), farthest Point : None
Centroid : (25, 463), farthest Point : (21, 467)
Centroid : (25, 463), farthest Point : (21, 467)
Centroid : (620, 455), farthest Point : None
Centroid : (620, 455), farthest Point : None
Centroid : (611, 375), farthest Point : None
Centroid : (611, 375), farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (319, 426), farthest Point : (329, 423)
Centroid : (319, 426), farthest Point : (329, 423)
Centroid : (395, 466), farthest Point : (377, 473)
Centroid : (395, 466), farthest Point : (377, 473)
Centroid : (313, 465), farthest Point : (327, 474)
Centroid : (313, 465), farthest Point : (327, 474)
Centroid : (389, 452), farthest Point : (413, 476)
Centroid : (389, 452), farthest Point : (413, 476)
Centroid : (376, 468), farthest Point : (391, 477)
Centroid : (602, 

Centroid : (96, 268), farthest Point : (97, 274)
Centroid : (68, 472), farthest Point : (73, 474)
Centroid : (68, 472), farthest Point : (73, 474)
Centroid : (532, 72), farthest Point : (547, 69)
Centroid : (532, 72), farthest Point : (547, 69)
Centroid : (30, 270), farthest Point : (34, 271)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (289, 203), farthest Point : (0, 479)
Centroid : (289, 203), farthest Point : (0, 479)
Centroid : (68, 473), farthest Point : (70, 479)
Centroid : (68, 473), farthest Point : (70, 479)
Centroid : (591, 154), farthest Point : (612, 150)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (285, 188), farthest Point : (0, 479)
Centroid : (285, 188), farthest Point : (0, 479)
Centroid : (172, 294), farthest Point : (190, 290)
Centroid : (172, 294), farthest Point : (190, 290)
Centroid : (53, 366), far

Centroid : None, farthest Point : None
Centroid : (24, 425), farthest Point : (12, 453)
Centroid : (24, 431), farthest Point : (13, 446)
Centroid : (23, 434), farthest Point : (13, 446)
Centroid : (23, 430), farthest Point : (16, 409)
Centroid : (23, 430), farthest Point : (16, 409)
Centroid : (22, 435), farthest Point : (10, 471)
Centroid : (233, 345), farthest Point : (248, 342)
Centroid : (24, 430), farthest Point : (16, 452)
Centroid : (24, 430), farthest Point : (16, 452)
Centroid : (24, 427), farthest Point : (13, 441)
Centroid : (24, 424), farthest Point : (23, 406)
Centroid : (24, 428), farthest Point : (23, 410)
Centroid : (22, 415), farthest Point : (15, 430)
Centroid : (23, 434), farthest Point : (13, 446)
Centroid : (23, 434), farthest Point : (13, 446)
Centroid : (16, 445), farthest Point : (19, 445)
Centroid : (23, 433), farthest Point : (23, 409)
Centroid : (22, 428), farthest Point : (35, 429)
Centroid : (22, 428), farthest Point : (35, 429)
Centroid : (22, 437), farthe

Centroid : (470, 356), farthest Point : (222, 266)
Centroid : (471, 357), farthest Point : (220, 280)
Centroid : (470, 357), farthest Point : (219, 286)
Centroid : (470, 357), farthest Point : (219, 286)
Centroid : None, farthest Point : None
Centroid : (473, 348), farthest Point : (228, 244)
Centroid : (473, 342), farthest Point : (228, 227)
Centroid : (473, 342), farthest Point : (228, 227)
Centroid : (7, 220), farthest Point : (2, 205)
Centroid : (473, 344), farthest Point : (231, 225)
Centroid : (473, 349), farthest Point : (224, 252)
Centroid : (473, 349), farthest Point : (224, 252)
Centroid : (472, 351), farthest Point : (221, 262)
Centroid : (472, 349), farthest Point : (220, 258)
Centroid : (473, 344), farthest Point : (220, 243)
Centroid : None, farthest Point : None
Centroid : (475, 205), farthest Point : (477, 202)
Centroid : (475, 205), farthest Point : (477, 202)
Centroid : (470, 330), farthest Point : (220, 211)
Centroid : (471, 338), farthest Point : (219, 244)
Centroid

Centroid : (534, 181), farthest Point : (550, 178)
Centroid : (509, 324), farthest Point : (314, 175)
Centroid : (160, 368), farthest Point : (175, 365)
Centroid : (160, 368), farthest Point : (175, 365)
Centroid : (126, 474), farthest Point : (141, 479)
Centroid : (507, 337), farthest Point : (293, 234)
Centroid : (519, 204), farthest Point : (504, 207)
Centroid : (171, 379), farthest Point : (186, 376)
Centroid : (148, 372), farthest Point : (163, 369)
Centroid : (148, 372), farthest Point : (163, 369)
Centroid : (511, 146), farthest Point : (519, 175)
Centroid : (636, 475), farthest Point : None
Centroid : (504, 324), farthest Point : (292, 242)
Centroid : (505, 336), farthest Point : (300, 230)
Centroid : (505, 336), farthest Point : (300, 230)
Centroid : (505, 340), farthest Point : (306, 218)
Centroid : (507, 334), farthest Point : (331, 177)
Centroid : (521, 207), farthest Point : (536, 200)
Centroid : (508, 331), farthest Point : (323, 173)
Centroid : (149, 387), farthest Point

Centroid : None, farthest Point : None
Centroid : (593, 450), farthest Point : (598, 451)
Centroid : (593, 450), farthest Point : (598, 451)
Centroid : (634, 468), farthest Point : (627, 470)
Centroid : (634, 468), farthest Point : (627, 470)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (480, 477), farthest Point : (482, 476)
Centroid : (480, 441), farthest Point : (481, 448)
Centroid : (480, 441), farthest Point : (481, 448)
Centroid : (565, 464), farthest Point : (569, 464)
Centroid : (565, 464), farthest Point : (569, 464)
Centroid : (540, 470), farthest Point : None
Centroid : (540, 470), farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (380, 476), farthest Point : (385, 474)
Centroid :

Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : None, farthest Point : None
Centroid : (76, 478), farthest Point : (81, 477)
Centroid : (76, 478), farthest Point : (81, 477)
Centroid : (315, 465), farthest Point : (291, 471)
Centroid : (315, 465), farthest Point : (291, 471)
Centroid : (357, 474), farthest Point : (371, 478)
Centroid : (357, 474), farthest Point : (371, 478)
Centroid : None, farthest Point : None
Centroid : None, farthest Point : N

In [ ]:
"""
Hand gesture recognition
==========================
This is main function for the project.
Source Code:
https://github.com/RobinCPC/CE264-Computer_Vision
Usage:
------
    gesture_hci.py [<video source>] (default: 0)
Keys:
-----
    ESC     - exit
    c       - toggle mouse control (default: False)
    t       - toggle hand tracking (default: False)
    s       - toggle skin calibration (need debug)
"""

import cv2
import numpy as np
import math
import time

# for controlling mouse and keyboard
import pyautogui
import sys

# Fail-safe mode (prevent from ou of control)
pyautogui.FAILSAFE = True
pyautogui.PAUSE = 0.1       # pause each pyautogui function 0.1 sec


# Build fixed-length Queue
class FixedQueue:
    """
    A container with First-In-First-Out (FIFO) queuing policy
    But can only storage maximum number of items in container
    """
    def __init__(self):
        self.list = []
        self.max_item = 5
        self.n_maj = 4

    def __str__(self):
        """Return list as string for printing"""
        return str(self.list)

    def push(self, item):
        """Enqueue the item into the queue, but check length before add in"""
        if self.list.__len__() == self.max_item:
            self.list.pop()
        self.list.insert(0, item)

    def pop(self):
        """Dequeue the earliest enqueued item still in the queue."""
        return self.list.pop()

    def major(self):
        """Return the number that shows often in list"""
        maj = 0
        count = 0
        for i in xrange(5):
            cur_cnt = self.list.count(i)
            if cur_cnt > count:
                maj = i
                count = cur_cnt
        return maj

    def count(self, value):
        """Return how many times that value show up in the queue"""
        return self.list.count(value)

    def isEmpty(self):
        """Return true if the queue is empty"""
        return len(self.list) == 0


# Dummy callback for trackbar
def nothing(x):
    pass

# uncomment if want to do on-line skin calibration
cv2.namedWindow('YRB_calib')
cv2.createTrackbar('Ymin', 'YRB_calib', 54, 255, nothing)
cv2.createTrackbar('Ymax', 'YRB_calib', 143, 255, nothing)
cv2.createTrackbar('CRmin', 'YRB_calib', 131, 255, nothing)
cv2.createTrackbar('CRmax', 'YRB_calib', 157, 255, nothing)
cv2.createTrackbar('CBmin', 'YRB_calib', 110, 255, nothing)
cv2.createTrackbar('CBmax', 'YRB_calib', 155, 255, nothing)

# Main part of gesture_hci
class App(object):
    def __init__(self, video_src):
        self.cam = cv2.VideoCapture(video_src)
        ret, self.frame = self.cam.read()
        cv2.namedWindow('gesture_hci')
        
        # set channel range of skin detection 
        self.mask_lower_yrb = np.array([44, 131, 80])       # [54, 131, 110]
        self.mask_upper_yrb = np.array([163, 157, 155])     # [163, 157, 135]
        # create trackbar for skin calibration
        self.calib_switch = False

        # create background subtractor 
        self.fgbg = cv2.BackgroundSubtractorMOG2(history=120, varThreshold=50, bShadowDetection=True)

        # define dynamic ROI area
        self.ROIx, self.ROIy = 200, 200
        self.track_switch = False
        # record previous positions of the centroid of ROI
        self.preCX = None
        self.preCY = None

        # A queue to record last couple gesture command
        self.last_cmds = FixedQueue()
        
        # prepare some data for detecting single-finger gesture  
        self.fin1 = cv2.imread('./test_data/index1.jpg')
        self.fin2 = cv2.imread('./test_data/index2.jpg')
        self.fin3 = cv2.imread('./test_data/index3.jpg')

        # switch to turn on mouse input control
        self.cmd_switch = False
        
        # count loop (frame), for debugging
        self.n_frame = 0


# On-line Calibration for skin detection (bug, not stable)
    def skin_calib(self, raw_yrb):
        mask_skin = cv2.inRange(raw_yrb, self.mask_lower_yrb, self.mask_upper_yrb)
        cal_skin = cv2.bitwise_and(raw_yrb, raw_yrb, mask=mask_skin)
        cv2.imshow('YRB_calib', cal_skin)
        k = cv2.waitKey(5) & 0xFF
        if k == ord('s'):
            self.calib_switch = False
            cv2.destroyWindow('YRB_calib')

        ymin = cv2.getTrackbarPos('Ymin', 'YRB_calib')
        ymax = cv2.getTrackbarPos('Ymax', 'YRB_calib')
        rmin = cv2.getTrackbarPos('CRmin', 'YRB_calib')
        rmax = cv2.getTrackbarPos('CRmax', 'YRB_calib')
        bmin = cv2.getTrackbarPos('CBmin', 'YRB_calib')
        bmax = cv2.getTrackbarPos('CBmax', 'YRB_calib')
        self.mask_lower_yrb = np.array([ymin, rmin, bmin])
        self.mask_upper_yrb = np.array([ymax, rmax, bmax])


# Do skin detection with some filtering
    def skin_detect(self, raw_yrb, img_src):
        # use median blurring to remove signal noise in YCRCB domain
        raw_yrb = cv2.medianBlur(raw_yrb, 5)
        mask_skin = cv2.inRange(raw_yrb, self.mask_lower_yrb, self.mask_upper_yrb)

        # morphological transform to remove unwanted part
        kernel = np.ones((5, 5), np.uint8)
        #mask_skin = cv2.morphologyEx(mask_skin, cv2.MORPH_OPEN, kernel)
        mask_skin = cv2.dilate(mask_skin, kernel, iterations=2)

        res_skin = cv2.bitwise_and(img_src, img_src, mask=mask_skin)
        #res_skin_dn = cv2.fastNlMeansDenoisingColored(res_skin, None, 10, 10, 7,21)

        return res_skin


# Do background subtraction with some filtering
    def background_subtract(self, img_src):
        fgmask = self.fgbg.apply(cv2.GaussianBlur(img_src, (25, 25), 0))
        kernel = np.ones((5, 5), np.uint8)
        fgmask = cv2.dilate(fgmask, kernel, iterations=2)
        #fgmask = self.fgbg.apply(cv2.medianBlur(img_src, 11))
        org_fg = cv2.bitwise_and(img_src, img_src, mask=fgmask)
        return org_fg

# Update Position of ROI
    def update_ROI(self, img_src):
        # setting flexible ROI range
        Rxmin,Rymin,Rxmax,Rymax = (0,)*4
        if self.ROIx - 100 < 0:
            Rxmin = 0
        else:
            Rxmin = self.ROIx - 100
        
        if self.ROIx + 100 > img_src.shape[0]:
            Rxmax = img_src.shape[0]
        else:
            Rxmax = self.ROIx + 100
        
        if self.ROIy - 100 < 0:
            Rymin = 0
        else:
            Rymin = self.ROIy - 100
        
        if self.ROIy + 100 > img_src.shape[1]:
            Rymax = img_src.shape[1]
        else:
            Rymax = self.ROIy + 100
        
        return Rxmin, Rymin, Rxmax, Rymax


# Find contour and track hand inside ROI
    def find_contour(self, img_src, Rxmin, Rymin, Rxmax, Rymax):
        cv2.rectangle(img_src, (Rxmax, Rymax), (Rxmin, Rymin), (0, 255, 0), 0)
        crop_res = img_src[Rymin: Rymax, Rxmin:Rxmax]
        grey = cv2.cvtColor(crop_res, cv2.COLOR_BGR2GRAY)

        _, thresh1 = cv2.threshold(grey, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        cv2.imshow('Thresh', thresh1)
        contours, hierchy = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

        # draw contour on threshold image
        if len(contours) > 0:
            cv2.drawContours(thresh1, contours, -1, (0, 255, 0), 3)
            
        return contours, crop_res


# Check ConvexHull  and Convexity Defects
    def get_defects(self, cnt, drawing):
        defects = None        
        hull = cv2.convexHull(cnt)
        cv2.drawContours(drawing, [cnt], 0, (0, 255, 0), 0)
        cv2.drawContours(drawing, [hull], 0, (0, 0, 255), 0)
        hull = cv2.convexHull(cnt, returnPoints=False)       # For finding defects
        if hull.size > 2:
            defects = cv2.convexityDefects(cnt, hull)        
        
        return defects


# Gesture Recognition
    def gesture_recognize(self, cnt, defects, count_defects, crop_res):
        # use angle between start, end, defect to recognize # of finger show up
        if type(defects) is not None and cv2.contourArea(cnt) >= 5000:
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(cnt[s][0])
                end = tuple(cnt[e][0])
                far = tuple(cnt[f][0])
                a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 180/math.pi
                if angle <= 90:
                    count_defects += 1
                    cv2.circle(crop_res, far, 5, [0, 0, 255], -1)
                cv2.line(crop_res, start, end, [0, 255, 0], 2)
                
        ## single fingertip check
        if count_defects == 0 and cv2.contourArea(cnt) >= 5000:
            count_defects = self.single_finger_check(cnt)
        
        # return the result of gesture recognition
        return count_defects


# Check if single-finger show up (OpenCV API using matchShape)
    def single_finger_check(self, cnt):
        # use single finger image to check current fame has single finger
        grey_fin1 = cv2.cvtColor(self.fin1, cv2.COLOR_BGR2GRAY)
        _, thresh_fin1 = cv2.threshold(grey_fin1, 127, 255, 0)
        contour_fin1, hierarchy = cv2.findContours(thresh_fin1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        cnt1 = contour_fin1[0]
        ret1 = cv2.matchShapes(cnt, cnt1, 1, 0)
        
        grey_fin2 = cv2.cvtColor(self.fin2, cv2.COLOR_BGR2GRAY)
        _, thresh_fin2 = cv2.threshold(grey_fin2, 127, 255, 0)
        contour_fin2, hierarchy = cv2.findContours(thresh_fin2.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        cnt2 = contour_fin2[0]
        ret2 = cv2.matchShapes(cnt, cnt2, 1, 0)
        
        grey_fin3 = cv2.cvtColor(self.fin3, cv2.COLOR_BGR2GRAY)
        _, thresh_fin3 = cv2.threshold(grey_fin3, 127, 255, 0)
        contour_fin3, hierarchy = cv2.findContours(thresh_fin3.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        cnt3 = contour_fin3[0]
        ret3 = cv2.matchShapes(cnt, cnt3, 1, 0)
        reta = (ret1 + ret2 + ret3)/3
        if reta <= 0.3:
            return 5        # set as one-finger module
        else:
            return 0        # not detect, still 0


# Use PyAutoGUI to control mouse event
    def input_control(self, count_defects, img_src):
        # update position difference with previous frame (for move mouse)
        d_x, d_y = 0, 0
        if self.preCX is not None:
            d_x = self.ROIx - self.preCX
            d_y = self.ROIy - self.preCY
        
        # checking current command, and filter out unstable hand gesture
        cur_cmd = 0
        if self.cmd_switch:
            if self.last_cmds.count(count_defects) >= self.last_cmds.n_maj:
                cur_cmd = count_defects
                #print 'major command is ', cur_cmd
            else:
                cur_cmd = 0     # self.last_cmds.major()
        else:
            cur_cmd = count_defects
        
        # send mouse input event depend on hand gesture
        if cur_cmd == 1:
            str1 = '2, move mouse dx,dy = ' + str(d_x*3) + ', ' + str(d_y*3)
            cv2.putText(img_src, str1, (50, 50), cv2.FONT_HERSHEY_TRIPLEX, 2, (0, 0, 255), 2)
            if self.cmd_switch:
                pyautogui.moveRel(d_x*3, d_y*3)
                self.last_cmds.push(count_defects)
                #pyautogui.mouseDown(button='left')
                #pyautogui.moveRel(d_x, d_y)
            #else:
            #    pyautogui.mouseUp(button='left')
        elif cur_cmd == 2:
            cv2.putText(img_src, '3 Left (rotate)', (50, 50), cv2.FONT_HERSHEY_TRIPLEX, 2, (0, 0, 255), 2)
            if self.cmd_switch:
                pyautogui.dragRel(d_x, d_y, button='left')
                self.last_cmds.push(count_defects)
                #pyautogui.scroll(d_y,pause=0.2) 
        elif cur_cmd == 3:
            cv2.putText(img_src, '4 middle (zoom)', (50, 50), cv2.FONT_HERSHEY_TRIPLEX, 2, (0, 0, 255), 2)
            if self.cmd_switch:
                pyautogui.dragRel(d_x, d_y, button='middle')
                self.last_cmds.push(count_defects)
        elif cur_cmd == 4:
            cv2.putText(img_src, '5 right (pan)', (50, 50), cv2.FONT_HERSHEY_TRIPLEX, 2, (0, 0, 255), 2)
            if self.cmd_switch:
                pyautogui.dragRel(d_x, d_y, button='right')
                self.last_cmds.push(count_defects)
        elif cur_cmd == 5:
            cv2.putText(img_src, '1 fingertip show up', (50, 50), cv2.FONT_HERSHEY_TRIPLEX, 2, (0, 0, 255), 2)
            if self.cmd_switch:
                self.last_cmds.push(count_defects)
        else:
            cv2.putText(img_src, 'No finger detect!', (50, 50), cv2.FONT_HERSHEY_TRIPLEX, 2, (0, 0, 255), 2)
            if self.cmd_switch:
                self.last_cmds.push(count_defects)  # no finger detect or wrong gesture


# testing pyautogui
    def test_auto_gui(self):
        if self.cmd_switch:
            # Drag mouse to control some object on screen (such as google map at webpage)
            distance = 100.
            while distance > 0:
                pyautogui.dragRel(distance, 0, duration=2, button='left')    # move right
                distance -= 25
                pyautogui.dragRel(0, distance, duration=2, button='left')    # move down
                distance -= 25
                pyautogui.dragRel(-distance, 0, duration=2, button='left')    # move right
                distance -= 25
                pyautogui.dragRel(0, -distance, duration=2, button='left')    # move down
                distance -= 25

            # scroll mouse wheel (zoom in and zoom out google map)
            pyautogui.scroll(10, pause=1.)
            pyautogui.scroll(-10, pause=1)

            pyautogui.scroll(10, pause=1.)
            pyautogui.scroll(-10, pause=1)

            # message box
            pyautogui.alert(text='pyautogui testing over, click ok to end', title='Alert', button='OK')
            self.cmd_switch = not self.cmd_switch   # turn off

# main function of the project (run all processes)
    def run(self):
        while self.cam.isOpened():
            if self.n_frame == 0:
                ini_time = time.time()
            ret, self.frame = self.cam.read()
            org_vis = self.frame.copy()
            #org_vis = cv2.fastNlMeansDenoisingColored(self.frame, None, 10,10,7,21) # try to denoise but time comsuming

            ### Skin detect filter
            yrb = cv2.cvtColor(self.frame, cv2.COLOR_BGR2YCR_CB)
            res_skin = self.skin_detect(yrb, org_vis)

            ## check if want to do skin calibration
            if self.calib_switch:
                self.skin_calib(yrb)
            
            ### Background Subtraction
            org_fg = self.background_subtract(org_vis)

            ### Find Contours and track hand inside ROI
            Rxmin, Rymin, Rxmax, Rymax = self.update_ROI(org_fg)
            contours, crop_res = self.find_contour(org_fg, Rxmin, Rymin, Rxmax, Rymax)

            ### Get Convexity Defects if Contour in ROI is bigger enough 
            drawing = np.zeros(crop_res.shape, np.uint8)
            max_area = -1
            ci = 0
            if len(contours) > 0:
                for i in range(len(contours)):
                    cnt = contours[i]
                    area = cv2.contourArea(cnt)
                    if area > max_area:
                        max_area = area
                        ci = i
                cnt = contours[ci]

                # use minimum rectangle to crop contour for faster gesture checking
                x, y, w, h = cv2.boundingRect(cnt)
                cv2.rectangle(crop_res, (x, y), (x+w, y+h), (0, 0, 255), 0)

                # check if start to track hand
                if self.track_switch:
                    M = cv2.moments(cnt)
                    if M['m00'] != 0:
                        self.ROIx = int(M['m10']/M['m00']) + Rxmin
                        self.ROIy = int(M['m01']/M['m00']) + Rymin - 30
                    else:
                        self.ROIx = 200
                        self.ROIy = 200

                # debug draw a  circle at center
                M = cv2.moments(cnt)
                if M['m00'] != 0:
                    cx = int(M['m10']/M['m00'])
                    cy = int(M['m01']/M['m00'])
                    cv2.circle(org_fg, (cx+Rxmin, cy+Rymin), 10, [0, 255, 255], -1)
                
                ### Check ConvexHull  and Convexity Defects
                defects = self.get_defects(cnt, drawing)

                ### Gesture Recognition
                count_defects = 0
                count_defects = self.gesture_recognize(cnt, defects, count_defects, crop_res)

                ### Input Control (Mouse Event)
                self.input_control(count_defects, org_fg)

                # update center position of ROI for next frame
                self.preCX = self.ROIx
                self.preCY = self.ROIy

            ### Display Image
            #cv2.imshow('original_view', org_vis)
            #cv2.imshow('YCR_CB', yrb)
            #cv2.imshow('YRB_skin', res_skin)
            #cv2.imshow('fgmask', fgmask)
            cv2.imshow('gesture_hci', org_fg)         # final result shows Here

            all_img = np.hstack((drawing, crop_res))
            cv2.imshow('Contours', all_img)

            #self.test_auto_gui()

            ch = cv2.waitKey(5) & 0xFF
            if ch == 27:
                break
            elif ch == ord('c'):
                self.cmd_switch = not self.cmd_switch
            elif ch == ord('s'):
                self.calib_switch = not self.calib_switch
            elif ch == ord('t'):
                self.track_switch = not self.track_switch

            if self.n_frame == 3:
                cur_time = time.time()
                #print 'time for one loop:',(cur_time - ini_time)
            self.n_frame = (self.n_frame + 1) % 4

        cv2.destroyAllWindows()


if __name__ == '__main__':
    # main function start here
    try:
        video_src = sys.argv[1]
    except:
        video_src = 0
    print(__doc__)
    App(0).run()


Hand gesture recognition
This is main function for the project.
Source Code:
https://github.com/RobinCPC/CE264-Computer_Vision
Usage:
------
    gesture_hci.py [<video source>] (default: 0)
Keys:
-----
    ESC     - exit
    c       - toggle mouse control (default: False)
    t       - toggle hand tracking (default: False)
    s       - toggle skin calibration (need debug)

